## 0. Importación y ejecución del entorno

En este notebook se ilustra un flujo RAG (Retrieval-Augmented Generation) usando:
- Carga de documentos: `langchain_community.document_loaders` (PDF/text).
- Separación de texto en chunks: `langchain_text_splitters`.
- Generación de embeddings: `langchain_ollama` (Ollama Embeddings).
- Almacenamiento vectorial: `langchain_chroma` (Chroma).
- LLM para generación: `langchain_google_genai` (Gemini) como ejemplo; también se muestra cómo configurar otros LLMs.

Objetivos de la sección: explicar las dependencias, comprobar las importaciones y dejar claro cómo configurar la variable de entorno `API_KEY` para los LLMs.

Notas prácticas:
- Si no tienes el PDF de ejemplo, coloca un archivo llamado `documento_ejemplo.pdf` o modifica la ruta en la celda de carga.
- Para reproducibilidad conviene crear un `persist_directory` cuando se crea la colección Chroma (ver sección 4).
- Antes de ejecutar las celdas que usan la API pública, verifica que `API_KEY` esté definida en el entorno o en un archivo `.env` (se usa `python-dotenv` en el notebook).

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, PDFPlumberLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import os

from dotenv import load_dotenv
load_dotenv()

api_key =os.getenv("API_KEY")

## 1. Implementación práctica de RAG

Esta sección contiene el flujo mínimo para montar RAG: carga del documento, fragmentación en chunks, generación de embeddings, construcción del vectorstore (Chroma) y consulta con un LLM que reescribe o responde usando el contexto recuperado.

Qué hace el código adjunto:
- `cargar_pdf(ruta_pdf)`: carga un PDF y devuelve una lista de `Document` por página (usa `PyPDFLoader`).
- `partir_texto(docs_or_text)`: normaliza y fragmenta el texto usando `CharacterTextSplitter` (parámetros por defecto: `chunk_size=100`, `chunk_overlap=20`).
- `crear_embeddings()`: instancia `OllamaEmbeddings` (modelo por defecto `nomic-embed-text`).
- `crear_vectorstore(docs, embedding)`: crea una colección Chroma en memoria; recomendamos añadir `persist_directory` para guardarla entre sesiones.
- `recuperar_contexto(pregunta, vectorstore)`: busca los 3 documentos más similares y concatena su contenido como contexto para el LLM.
- `responder_llm(pregunta, contexto)`: envía un prompt al LLM y devuelve la respuesta (ej.: Gemini).

Limitaciones y recomendaciones prácticas:
- Asegúrate de usar un documento de ejemplo (p. ej. `documento_ejemplo.pdf`) en el repo o cambia la ruta.
- Para mejorar trazabilidad, guarda `page_content` y `metadata` de cada chunk en Chroma con un `chunk_id`.
- Mantén el `temperature` del LLM bajo (0.0–0.3) para respuestas basadas en contexto, y pide al LLM que cite fragmentos si necesitas fuentes.

Siguiente paso en el notebook: se mostrarán experimentos para comparar embeddings y LLMs, medir efectos del splitter y ejemplos de prompts refinados.

In [ ]:
def cargar_pdf(ruta_pdf: str):
    loader = PyPDFLoader(ruta_pdf)
    docs = loader.load()  # documentos por página con metadata
    return docs

text = cargar_pdf("seguridad.pdf")

def partir_texto(docs_or_text):
    splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
    
    # aceptar tanto lista de Documents como texto plano
    if isinstance(docs_or_text, list):
        texts = [d.page_content for d in docs_or_text]
    else:
        texts = [docs_or_text]
        
    docs = splitter.create_documents(texts)
    return docs

docs = partir_texto(text)

def crear_embeddings():
    embedding = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)


embedding = crear_embeddings()

def crear_vectorstore(docs, embedding):
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embedding,
        collection_name="base_rag_mem_384"
    )
    return vectorstore

vectorstore = crear_vectorstore(docs, embedding)

def recuperar_contexto(pregunta, vectorstore):
    docs_rel = vectorstore.similarity_search(pregunta, k=3)
    contexto = "\n".join([d.page_content for d in docs_rel])
    return contexto

pregunta = "¿Que es el Command Injection?"
contexto = recuperar_contexto(pregunta, vectorstore)

def responder_llm(pregunta, contexto):
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.3,
        api_key=os.getenv("API_KEY")
    )
    prompt = (
        f"Eres un asistente sobre LangChain.\n"
        f"Responde usando sólo este contexto:\n{contexto}\n"
        f"Pregunta: {pregunta}"
    )
    respuesta = llm.invoke(prompt)
    return respuesta.content

respuesta = responder_llm(pregunta, contexto)
print(respuesta)


## 2. Exploración de embeddings y LLMs (breve)

Qué probar (rápido):
- Generar embeddings con 2–3 modelos (ej. `nomic-embed-text`, `all-minilm`).
- Crear colecciones Chroma separadas y buscar con 3 preguntas simples.
- Medir `recall@k` (k=1,3) y tiempo de respuesta.

Consejo: guarda los resultados en un dict o CSV para comparar fácilmente. Si un modelo no está disponible, cámbialo por uno existente.

In [ ]:
#Varios embeddings
embedding_models = ["nomic-embed-text", "mxbai-embed-large", "all-minilm"]

def probar_embeddings(pregunta):
    resultados = {}
    for model in embedding_models:
        emb = OllamaEmbeddings(model=model)
        vs = Chroma.from_documents(docs, emb, collection_name=f"prueba_{model}")
        docs_rel = vs.similarity_search(pregunta, k=3)
        resultados[model] = [d.page_content for d in docs_rel]
    return resultados

res_emb = probar_embeddings("¿que es Injection (familia)?")
print(res_emb)


In [ ]:
llm_models = {
    "gemini-2.5-flash": ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2, api_key=os.getenv("API_KEY")),
    "gemini-2.5-pro": ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.2, api_key=os.getenv("API_KEY")),
}

def probar_llms(pregunta, contexto):
    resultados = {}
    for nombre, llm in llm_models.items():
        prompt = f"Contexto:\n{contexto}\nPregunta: {pregunta}"
        respuesta = llm.invoke(prompt)
        resultados[nombre] = respuesta.content
    return resultados

ctx = recuperar_contexto("Dime que es Cross-Site Scripting", vectorstore)
res_llm = probar_llms("Dime que es Cross-Site Scripting", ctx)
print(res_llm)


### 3. Optimización del separador (investigación)

Resumen de resultados y recomendaciones breves:
- CharacterTextSplitter: simple y efectivo como punto de partida. Recomendado para artículos y notas (chunk_size 80–150, overlap 10–20).
- RecursiveCharacterTextSplitter: mantiene estructura (secciones → párrafos → oraciones). Mejor para manuales o documentos largos (chunk_size 150–300, overlap 20–40).
- TokenTextSplitter: divide por tokens del modelo; útil si necesitas respetar límites de tokens del LLM o medir coste por token. Ideal cuando trabajas con límites estrictos.
- SentenceSplitter / split por oraciones: evita cortar frases; bueno para textos legales o narrativos donde la oración completa importa.
- Splitters basados en reglas (regex/Markdown): los mejores para código, README o tablas (ej.: MarkdownTextSplitter para respetar encabezados y bloques de código).

Conclusión práctica: elegir según el tipo de documento — para empezar usa Character/Recursive; para código o markdown usa splitters por estructura; si el modelo impone límites, usa TokenTextSplitter.

In [ ]:
def probar_splitter_sizes(text):
    tamaños = [(80, 10), (150, 20), (300, 40)]
    resultados = {}

    # Normalizar la entrada a una lista de strings que CharacterTextSplitter espera
    if isinstance(text, list):
        # Si es una lista de Document objetos, extraer page_content
        if len(text) > 0 and hasattr(text[0], "page_content"):
            texts = [d.page_content for d in text]
        else:
            texts = [str(t) for t in text]
    else:
        texts = [text]

    for size, overlap in tamaños:
        splitter = CharacterTextSplitter(chunk_size=size, chunk_overlap=overlap)
        docs = splitter.create_documents(texts)
        n_docs = len(docs) if docs else 0
        avg_len = sum(len(d.page_content) for d in docs)/n_docs if n_docs else 0
        resultados[(size, overlap)] = {"n_chunks": n_docs, "avg_len": avg_len}
    return resultados

split_stats = probar_splitter_sizes(text)
print(split_stats)


## 4. Gestión de bases vectoriales (investigación)

Comparación corta (ventajas / desventajas):
- Chroma: fácil de usar y buena persistencia local. Ventaja: integración rápida en notebooks. Desventaja: menos optimizada para escala masiva. Caso: prototipos y PoC.
- FAISS: muy rápida en búsquedas locales y flexible en índices. Ventaja: rendimiento. Desventaja: metadata y persistencia requieren manejo extra. Caso: sistemas locales que requieren velocidad.
- Qdrant: soporte nativo para metadata y filtros, API REST. Ventaja: filtrado y escalado moderado. Desventaja: requiere servicio. Caso: apps con filtrado por metadata.
- Milvus: diseñada para grandes volúmenes y escalabilidad. Ventaja: escala horizontal. Desventaja: complejidad operativa. Caso: grandes datasets en producción.
- Pinecone / Weaviate (gestionados): ventaja: servicio completo y features; desventaja: coste y latencia de red. Caso: producción sin gestionar infra.

Recomendación práctica: empezar con Chroma (persist_directory) y probar FAISS si necesitas más velocidad local; migrar a Qdrant/Milvus/Pinecone según necesidad de metadata y escala.

In [ ]:
def chunks_con_metadatos(docs):
    for i, d in enumerate(docs):
        d.metadata = {"chunk_id": i, "tema": "Command Injection"}
    return docs

docs_meta = chunks_con_metadatos(docs)

vect_meta = Chroma.from_documents(
    documents=docs_meta,
    embedding=embedding,
    collection_name="meta_rag"
)

filtrados = vect_meta.similarity_search("¿Command Injection?", k=3, filter={"tema": "Command Injection"})
for d in filtrados:
    print(d.metadata, d.page_content)


## 5. Refinamiento de prompts (rápido)

Tips sencillos:
- Pide respuestas cortas y pide fuentes: 'Responde en bullets y al final pon Fuentes'.
- Baja `temperature` (0.0–0.3) si quieres respuestas fieles al contexto.
- Si quieres parsear fácil: pide salida JSON con `answer` y `sources`.

Prueba: Ejecuta 2–3 prompts distintos con la misma pregunta y compara cuál da mejores fuentes.

In [ ]:
# Prompt con bullets
prompt_bullets = ChatPromptTemplate.from_messages([
    ("system", "Responde con bullets y cita fragmentos. Si no está, di 'No está en el documento'."),
    ("human", "Pregunta: {question}\nContexto:{context}\nFormato: - punto 1 [Fuente]")
])

def responder_bullets(pregunta, contexto):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, api_key=os.getenv("API_KEY"))
    formatted_prompt = f"{pregunta}\nContexto:{contexto}"
    return llm.invoke(formatted_prompt).content

print(responder_bullets("Explicame que es Insecure Direct Object Reference (IDOR)", contexto))


In [ ]:
# Prompt con verificación
prompt_verif = ChatPromptTemplate.from_messages([
    ("system", "Responde breve y agrega al final 'Verificado con contexto'."),
    ("human", "Pregunta: {question}\nContexto:{context}")
])

def responder_verif(pregunta, contexto):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.1, api_key=os.getenv("API_KEY"))
    formatted_prompt = f"{pregunta}\nContexto:{contexto}"
    return llm.invoke(formatted_prompt).content

print(responder_verif("Dime ejemplos sobre Insecure Direct Object Reference (IDOR)", contexto))


In [ ]:
# Prompt con fuentes
prompt_fuentes = ChatPromptTemplate.from_messages([
    ("system", "Al final incluye 'Fuentes' con los fragmentos usados."),
    ("human", "Pregunta: {question}\nContexto:{context}")
])

def responder_fuentes(pregunta, contexto):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2, api_key=os.getenv("API_KEY"))
    formatted_prompt = f"{pregunta}\nContexto:{contexto}"
    return llm.invoke(formatted_prompt).content

print(responder_fuentes("Resume el documento en 2 frases.", contexto))
